In [25]:
import pandas as pd
import keras
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv("unemployment-rate-men-vs-women.csv")

In [2]:
data.columns

Index(['Entity', 'Code', 'Year',
       'Unemployment, female (% of female labor force) (modeled ILO estimate)',
       'Unemployment, male (% of male labor force) (modeled ILO estimate)',
       'Population (historical estimates)', 'Continent'],
      dtype='object')

In [3]:
mapper = {'Unemployment, female (% of female labor force) (modeled ILO estimate)': 'unemployment_women',
          'Unemployment, male (% of male labor force) (modeled ILO estimate)': 'unemployment_men',
          'Population (historical estimates)': 'population'
          }

data.rename(columns=mapper, inplace=True)
data.rename(str.lower, axis='columns', inplace=True)


In [4]:
data.isna().sum()

type(data)

pandas.core.frame.DataFrame

In [13]:
# data['code'].sort_values('code').fillna(method='ffill', inplace=True)
# data['unemployment_men'].fillna(value=data['unemployment_men'].mean(), inplace=True)
# data['unemployment_women'].fillna(value=data['unemployment_women'].mean(), inplace=True)
# data['population'].fillna(value=data['population'].mean(), inplace=True)

def filler(dataframe: pd.DataFrame, column: str):
    """
    applies the forward fill method of the pandas package on a column in a dataframe

    ARGS:
    """
    dataframe[column] = dataframe[column].fillna(method='ffill')
    return dataframe[column]

def mean_filler(dataframe: pd.DataFrame, column: str):
    """
    applies mean of a column on all missing values in a column, do not use of missing values are too much
    """
    dataframe[column] = dataframe[column].fillna(value=dataframe[column].mean())
    return dataframe[column]

filler(data, "code")
filler(data, 'continent')
mean_filler(data, 'unemployment_men')
mean_filler(data, 'unemployment_women')
mean_filler(data, 'population')

data.to_csv("unemployment_rate.csv")

C:\Users\KingED\AppData\Local\Temp\ipykernel_22936\763551472.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe[column] = dataframe[column].fillna(method='ffill')


In [23]:
new = pd.read_csv("unemployment_rate.csv")